# 数据获取和存储

+ 在这一节中，我们主要介绍如何获取和读入数据，以及如何存储数据。
+ 其中涉及数据格式，网络数据抓取等内容。
+ 程序在3.3或更高版本中，如果在Python 2.7运行，可能会有所差别，特别是几个涉及中文字符的例子。

+ 下面是基本设定，需要导入的包和一些简单设定。

In [1]:
import os
import sys
import numpy as np
from numpy.random import randn
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt


np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

## Reading and Writing Data in Text Format

+ 读写文本格式的数据， 在所有pandas 包中关于读取的函数中个，read_csv 和read_table 是使用最多的。
+ 在读取存储在文本文件中的数据时，首先要关注一下 文档路径 以及 当前 notebook的工作目录

In [2]:
%pwd

'/Users/wrk/Documents/python_environment/DataAnalysisWithPython'

+ 打开文件，大致了解一下文档中数据的格式

In [3]:
#运用dos命令
!type data\ex1.csv  

/bin/sh: line 0: type: dataex1.csv: not found


+ 逗号分隔的数据，可以用read_csv函数读入

In [4]:
df = pd.read_csv('data/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 也可以用read_table读入，只是这时需要 设置 数据之间的分隔符参数： sep=','

In [5]:
pd.read_table('data/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 看看ex2文件，其中第一行不是变量名

In [6]:
!type data\ex2.csv

/bin/sh: line 0: type: dataex2.csv: not found


+ 因此需要指定和变量名相关的参数：header=None 或者 names=['a', 'b', 'c', 'd', 'message'] 等

In [7]:
pd.read_csv('data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('data/ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [9]:
pd.read_csv('data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 我们也可以指定DataFrame 的 行索引关键字：index_col='message'  说明用message这列作为行索引关键字
+ 也可以用index_col=4,message的列号指定其为关键字

In [10]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


+ 再看下面的例子

In [11]:
!type data\csv_mindex.csv

/bin/sh: line 0: type: datacsv_mindex.csv: not found


+ index关键字可以是两列，这样就有多层索引。

In [12]:
parsed = pd.read_csv('data/csv_mindex.csv', index_col=['key1', 'key2'])
print(parsed)
parsed.ix['one'].ix['b'].ix['value1']

           value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16


3

+ 分隔符可以有多种形式，比如如果分隔符为数量不等长空格符，则用正则表达式 '\s+'表示分隔符格式

In [13]:
list(open('data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [14]:
result = pd.read_table('data/ex3.txt', sep=',')
print(result)

result2 = pd.read_table('data/ex3.txt', sep='\s+')
print(result2)

               A         B         C
0  aaa -0.264438 -1.026059 -0.619500
1  bbb  0.927272  0.302904 -0.032399
2  ccc -0.264273 -0.386314 -0.217601
3  ddd -0.871858 -0.348382  1.100491
            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


+ pandas自动推断出第一行为列名，第一列为index. 

+ 有时候文档中会有说明文字，需要指定哪些行不用读入。
+ 选项 skiprows给出跳过的行。

In [15]:
!type data\ex4.csv

/bin/sh: line 0: type: dataex4.csv: not found


In [16]:
test1 = pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])
print(test1)
test2 = pd.read_csv('data/ex4.csv')
print(test2)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
                                                                      # hey!
a                                                  b        c   d    message
# just wanted to make things more difficult for... NaN      NaN NaN      NaN
# who reads CSV files with computers                anyway? NaN NaN      NaN
1                                                  2        3   4      hello
5                                                  6        7   8      world
9                                                  10       11  12       foo


+ pandas会自动处理常见的丢失值，不过也可以指定哪些值是缺失值
+ 我们甚至可以给不同列指定不同的缺失值。
+ 下面的例子中有标注了的缺失值 NA；也有没有标注的，连续两个逗号表示中间缺了一个数

In [17]:
!type data\ex5.csv

/bin/sh: line 0: type: dataex5.csv: not found


In [18]:
result = pd.read_csv('data/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 指定缺失值的实例

In [19]:
result = pd.read_csv('data/ex5.csv', na_values=['LoveLTT'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [21]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


+ 处理带中文的表格，用Python 3.3的同学看看这里是否有差别

In [22]:
Astock=pd.read_table("data/A.txt",encoding="gbk")
Astock.columns 

Index(['代码', '名称', '涨幅%%', '涨跌', '最高%%', '现价', '开盘%%', '现量', '投资收益', '市现率',
       '市净率', '市盈(动)', '每股未分配', '每股未分配.1', '昨收', '资产负债率%%', '人均持股', '净利润率%%',
       '细分行业', '流通股本(万)', '地区', '振幅%%', 'Unnamed: 22'],
      dtype='object')

### 逐块读入数据

当文件很大的时候，比如上百M。可能逐块读入更好。下面的例子有10000行数据

In [23]:
result=pd.read_csv('data/ex6.csv')
pd.read_csv('data/ex6.csv', nrows=5) #只读前5行

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 设置 chunksize实现逐块阅读
+ 我们可以通过.read　方法将前面若干行读出来。

In [24]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)
print(type(chunker))
chunker.read(5)

<class 'pandas.io.parsers.TextFileReader'>


,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 也可以用循环分块处理
+ 下面例子统计key列中各字母出现的频数，这里用到了Series求和的对齐功能。
+ .value_counts()是统计频数的方法

In [25]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:5]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
dtype: float64

**\color{red} end**

### 将数据写出到文本格式

首先看看前面处理过的csv文件

In [26]:
data = pd.read_csv('data/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 用数据data的方法to_csv写出到文件

In [27]:
data.to_csv('data/out.csv')
!type data\out.csv

/bin/sh: line 0: type: dataout.csv: not found


+ sys.stdout 是输出到屏幕，看先观察一下效果
+ sep='|' 分割符为 | 

In [28]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


+ na_rep='NULL' 表示用"NULL"代替 缺失值位置

In [29]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


+  index=False, header=False 说明不输出行和列索引

In [30]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


+ 当然也可以重新指定列名和索引

In [31]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


+ 下面是一个时间序列索引数据读写，我们会在后面专门介绍

In [32]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('data/tseries.csv')
!type data\tseries.csv

/bin/sh: line 0: type: datatseries.csv: not found


In [33]:
Series.from_csv('data/tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 从网络url中读

+ 网络url直接指向一个文本格式的文件：一个csv文件或者一个txt文件
+ 我们以从sina下载 高频数据，日线数据为例进行说明，这里要用到 urllib包。
+ 我们用 urlretrieve 函数读入存在本地，然后用read_table 读入数据
+ 可以直接用read_table读网络rul指向的文本格式文件，
   -  data= pd.read_table(url,na_values="--",encoding="gbk",parse_dates=True,index_col=0)
   -  不过出现问题，原因不明。


In [34]:
from urllib.request import urlretrieve
url = 'http://market.finance.sina.com.cn/downxls.php?date=2015-09-09&symbol=sh600540'  
urlretrieve(url, "demo.txt")
dataf= pd.read_table("demo.txt",na_values="--",encoding="gbk",parse_dates=True,index_col=0)
dataf[:8]
 

,成交价,价格变动,成交量(手),成交额(元),性质
成交时间,,,,,
2017-09-29 15:00:04,8.63,0.01,10,8630,中性盘
2017-09-29 14:59:59,8.62,NaN,517,445653,卖盘
2017-09-29 14:59:49,8.62,-0.03,212,182743,中性盘
2017-09-29 14:59:49,8.65,0.04,176,152240,买盘
2017-09-29 14:59:44,8.61,-0.03,763,656943,卖盘
2017-09-29 14:59:34,8.64,0.04,206,177984,买盘
2017-09-29 14:59:24,8.60,-0.04,10,8600,中性盘
2017-09-29 14:59:19,8.64,0.04,1135,980640,买盘


注意数据中没有日期，read_table自己加上的

### 读取 网络表格，

+ 很多网络数据都是以表格形式存在的，我们需要分析html文件，提取其中的表格
+  导入包和表格所在的url地址
```
from lxml.html import parse
form  urllib.request import urlopen
#url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/601899.phtml?year=2015&jidu=1'
url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/601899.phtml?year=2015&jidu=1'
```
+ 读取并提取所有表格
```
parsed = parse(urlopen(url))
doc = parsed.getroot()
tables=doc.findall('.//table')
```

+ 选定我们需要的表格

```
table = tables[19] 
```

+  找到表格中所有行

```
rows = table.findall('.//tr')

```
+ 网络表格一般都有标题行，即th单元格，数据行则是td单元格。用如下的函数提取数据单元中的数据
```
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts] 
```

如果kind ="th"，则是标题行中的数据，相当于变量名。

+ 需要首先观察网页中的表格，必要时需要看html原文件（确定所需表格是第几个表格是一定要看原文件），看看相应数据从第几行开始，然后将所有数据整合在一起生成DataFrame数据。

   - 读入第三行开始的所有行，生成一个list的list，其中内层的list代表一行数据
  
  ``` 
    data = [_unpack(r) for r in rows[2:]] 
   ```
   - 也可以用下面代码读入中文列名(变量名)
   
  ```
   header=_unpack(row[1],kind="td")
   ```
   - 整合为 DataFrame数据
   
   ```
   TextParser(data, names=header).get_chunk()
   ```

In [35]:
from lxml.html import parse
from urllib.request import urlopen
#from urllib2 import urlopen
#url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/601899.phtml?year=2015&jidu=1'
url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/601899.phtml?year=2015&jidu=1'


parsed = parse(urlopen(url))
doc = parsed.getroot()
tables=doc.findall('.//table')
##读取并提取所有表格


table = tables[19]
##rows = calls.findall('.//tr')


from pandas.io.parsers import TextParser


def _unpack(row, kind='td'):##针对某一行进行操作
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符


def parse_options_data(table):
    rows = table.findall('.//tr')
    header = ['date','open','high','close','low','vol','amount','ratio'] # 回避中文处理
    data = [_unpack(r) for r in rows[2:]]
    return TextParser(data, names=header).get_chunk()

datad = parse_options_data(table)

IndexError: list index out of range

In [217]:
datad[:5]

,date,open,high,close,low,vol,amount,ratio
0,2015-03-31,7.712,7.712,7.504,7.486,4.896260e+08,2.139222e+09,1.737
1,2015-03-30,7.660,7.799,7.729,7.469,5.973943e+08,2.610107e+09,1.737
2,2015-03-27,7.747,7.868,7.695,7.660,4.691807e+08,2.088989e+09,1.737
3,2015-03-26,7.834,7.834,7.660,7.486,7.119153e+08,3.128761e+09,1.737
4,2015-03-25,7.799,8.094,7.851,7.729,1.436269e+09,6.534120e+09,1.737


+ 其它解析html的例子，比如获得链接地址，在html中链接的标签是a
+ 获得具体链接字符串得用 .get 方法

In [ ]:
links=doc.findall('.//a')
lin=links[0]
lin.get('href')

In [ ]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-5:]

+   .text_content方法 可以得到标签内的文本

In [ ]:
cont=links[10].text_content() 
print(cont)
lin=links[10].get('href') 
print(lin)

## 二进制数据 格式  Binary data formats

+ Python 数据的二进制存储文档一般是带有 .pkl 扩展名的文件。 
+ 在pandas中，有读入数据的方法，而数据对象对象DataFrame和Series也有将数据写入二进制文件的方法。
+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [ ]:
frame = pd.read_csv('data/ex1.csv')
frame
frame.to_pickle('data/frame_pickle')

In [ ]:
pd.read_pickle('data/frame_pickle')

+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [ ]:
frame

In [ ]:
frmae=1

In [ ]:
import pickle
file1=open("data/pickle_dump.pkl","wb")
pickle.dump(frame,file=file1)
file1.close()
file1=open("data/pickle_dump.pkl","rb")
data1=pickle.load(file=file1)
file1.close()
!del data\pickle_dump.pkl  
data1

+ 文件必须关掉后才可以删除

## 数据库使用

+ 使用文本文件进行存储和读取的效率比较低，当数量两非常大时，需要考虑使用数据库。
+ 下面介绍使用SQLite数据库，其通过python的sqlite3驱动
+ python和数据库之间的交互非常简单，特别是pandas提供了一些函数，使我们能轻松进行数据库操作

命令|功能
------|------
import sqlite3 |导入数据库包
conn = sqlite3.connect('data/mydatabase.db')| 创建在硬盘上面
conn = sqlite3.connect('"memory:') | 创建在内存上面

+ 其中conn对象是数据库链接对象，而对于数据库链接对象来说，具有以下操作：

命令|功能
------|------
        commit()     |       事务提交
        rollback()    |     事务回滚
        close()       |     关闭一个数据库链接
        cursor()      |    创建一个游标
      cu = conn.cursor() |   这样我们就创建了一个游标对象：cu. 在sqlite3中，所有sql语句的执行都要在游标对象的参与下完成      
    对于游标对象cu，具有以下具体操:|
        execute()        |执行一条sql语句
        executemany()   |执行多条sql语句
        close()         |游标关闭
        fetchone()     |从结果中取出一条记录
        fetchmany()    |从结果中取出多条记录
        fetchall()   |从结果中取出所有记录
        scroll()   |游标滚动    
   

+ 下面的例子在内存中创建了一个数据库，并生成数据库的一个表

In [ ]:
import sqlite3

query = """
CREATE TABLE test0
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

+ 将数据插入相应的表,使用sqlite中的函数

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
print(data)
stmt = "INSERT INTO test0 VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

+ 我们可以使用pandas中的函数向数据库中插入一个表格

         import pandas.io.sql as sql

+ 在python2.7中，我们插入有中文关键字的DataFrame时需要
```
reload(sys)
sys.setdefaultencoding( "utf-8" )
```
+ to_sql函数可以插入一个DataFrame表格，注意如果表格存在，需要设置参数if_exists
    -  if_exists='fail' 如果存在就放弃插入
    -  if_exists='replace' 删掉原来的，创建新的表 
    -  if_exists='append' 插入数据

In [ ]:
import pandas.io.sql as sql
#reload(sys)
#sys.setdefaultencoding( "utf-8" )
sql.to_sql(dataf,"highfreq",con)
sql.to_sql(datad,"daily",con)

+ 从数据库中读取数据，用到了pandas中的函数

In [ ]:
mysql="SELECT * from highfreq LIMIT 5"

sql.read_sql(mysql,con)
sql.read_sql??

+ index_col 参数指定index

In [ ]:

sql.read_sql('select * from daily limit 5', con,index_col=["index","date"])



## 函数

编一个从sina抓取特定时间段内的日线数据的函数。注意和上面的细节上的不同。
 1. 股票复权与否
 2. 指数还是股票
 3. 复权数据多一列复权因子

In [ ]:
 
from lxml.html import parse
from urllib.request import urlopen
import pandas as pd

def get_sina_stock(stockid,start,end,isstock=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符    
    def parse_options_data(table,isstock):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] # 回避中文处理
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    for cq in daterange: 
        if isstock:
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(urlopen(url))
        except:
            print("download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3))
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    


In [ ]:
data1=get_sina_stock(601899,"1990-1-1","2015-8-15")
data2=get_sina_stock("000001","1990-1-1","2015-8-15",isstock=False)
data2.to_csv("data/000001.csv")
pd.concat([data1.head(3),data1.tail(3)])

In [ ]:
pd.concat([data2.head(3),data2.tail(3)])


## 分组作业

+ 每组选择一个行业，或者一个板块的所有股票，编程完成下面任务：
   - 从sina下载近5年的日线数据，将所有数据存为pandas的panel数据（复权和不复权各一个数据文件）
   - 从sina下载一周高频分时数据，将所有数据存为pandas的panel数据
   - 选择 （ 财务摘要，财务指标，资产负债表，利润表，或现金流量表）中的一个或多个表 近5年的数据，下载行业内所有股票构建一个Sqlite数据库    
     -  比如打开
        http://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/600030/displaytype/4.phtml ，
        在"财务数据"后面就可以看见相应的链接


     
